In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

plt.style.use('dark_background')

data_path = 'Data/co2_mm_gl.csv'
df = pd.read_csv(data_path, comment='#')

print(df.head())

In [ ]:
date_components = {
    'year': df['year'],
    'month': df['month'],
    'day': 1  # since data is monthly, we set day to 1
}
df['date'] = pd.to_datetime(date_components)

df.set_index('date', inplace=True)

In [ ]:
co2_series = df['average']
co2_series_decomposed = seasonal_decompose(co2_series, model='additive', period=12)
fig = co2_series_decomposed.plot()
plt.savefig('Plots/co2_decomposition_raw_output.png')
plt.show()

In [ ]:
# Raw data + Trend
fig, ax = plt.subplots(figsize=(12,6))
# plot raw data
ax.plot(co2_series.index, co2_series, label='Observed Monthly CO2', color='skyblue', alpha=0.8)
# plot trend component
ax.plot(co2_series_decomposed.trend.index, co2_series_decomposed.trend, label='Long-term Trend', color='orange', linewidth=2)
# add title and labels
ax.set_title('Mauna Loa CO2: Observed Monthly Levels vs. Long-term Trend', fontsize=16)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('CO2 Concentration (ppm)', fontsize=12)
# customize tick parameters
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
# add legend
legend = ax.legend(fontsize=10, facecolor='darkgray', edgecolor='white')
for text in legend.get_texts():
    text.set_color('black')
# add grid for readability
ax.grid(True, linestyle='--', alpha=0.5, color='darkgray')

plt.tight_layout()
plt.savefig('Plots/co2_observed_vs_trend.png', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
# plot seasonal component
ax.plot(co2_series_decomposed.seasonal.loc['1980':'1990'].index, 
        co2_series_decomposed.seasonal.loc['1980':'1990'], 
        color='magenta', linewidth=1.5)
# add title and labels
ax.set_title('Mauna Loa CO2: Monthly Seasonal Cycle', fontsize=14)
ax.set_xlabel('Date', fontsize=12, color='white')
ax.set_ylabel('Seasonal Variation (ppm)', fontsize=12, color='black')
# customize tick parameters
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
# add legend
legend = ax.legend(fontsize=10, facecolor='darkgray', edgecolor='white')
for text in legend.get_texts():
    text.set_color('white')
# add grid for readability
ax.grid(True, linestyle='--', alpha=0.5, color='darkgray')

plt.tight_layout()
plt.savefig('Plots/co2_seasonal_cycle.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_temp = pd.read_csv('Data/GLB.Ts+dSST.csv', comment='#', na_values='***', index_col=False)
print(df_temp.head())

In [ ]:
month_columns = df_temp.columns[1:13]
df_long = df_temp.melt(id_vars=['Year'], value_vars=month_columns, var_name='Month_Name', value_name='Anomaly_C')

# clean up
df_long['Anomaly_C'] = pd.to_numeric(df_long['Anomaly_C'], errors='coerce')
df_long.dropna(subset=['Anomaly_C'], inplace=True)

# create datetime index
month_mapping = {name: i for i, name in enumerate(month_columns, start=1)}
df_long['Month'] = df_long['Month_Name'].map(month_mapping)
date_components_temp = {
    'Year': df_long['Year'], 
    'Month': df_long['Month'], 
    'Day': 1
}
df_long['Date'] = pd.to_datetime(date_components_temp)

# Filter to start from 1980 to match CO2 data
start_date = '1980-01-01'
df_long_filtered = df_long[df_long['Date'] >= start_date].copy()
df_temp_series = df_long_filtered.set_index('Date')['Anomaly_C']

temp_series_decomposed = seasonal_decompose(df_temp_series, model='additive', period=12)
temp_trend_for_plot = temp_series_decomposed.trend

co2_trend_for_plot = co2_series_decomposed.trend[co2_series_decomposed.trend.index >= start_date]
common_index = co2_trend_for_plot.index.intersection(temp_trend_for_plot.index)

co2_final_trend = co2_trend_for_plot.loc[common_index]
temp_final_trend = temp_trend_for_plot.loc[common_index]

co2_final_trend = co2_final_trend.dropna()
temp_final_trend = temp_final_trend.dropna()

final_common_index = co2_final_trend.index.intersection(temp_final_trend.index)
co2_final_trend = co2_final_trend.loc[final_common_index]
temp_final_trend = temp_final_trend.loc[final_common_index]

# Print heads to confirm they are clean and start/end correctly
print("CO2 Final Trend Head:")
print(co2_final_trend.head())
print("\nTemperature Final Trend Head:")
print(temp_final_trend.head())




In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
# axis 1 - CO2 Trend
color_co2 = 'orange'
ax1.set_xlabel('Year', color='white')
ax1.set_ylabel('CO2 Concentration (ppm)', color=color_co2)
ax1.plot(co2_series_decomposed.trend.index, co2_series_decomposed.trend,
         color=color_co2, linewidth=3, label='CO2 Trend (ppm)')
ax1.tick_params(axis='x', colors='white')
ax1.tick_params(axis='y', labelcolor=color_co2)

# axis 2 - Temperature
ax2 = ax1.twinx()
color_temp = 'red'
ax2.set_ylabel('Temperature Anomaly (°C)', color=color_temp)
ax2.plot(temp_trend_for_plot.index, temp_trend_for_plot, 
         color=color_temp, linewidth=3, linestyle='--', label='Temperature Anomaly Trend (°C)')
ax2.tick_params(axis='y', labelcolor=color_temp)

fig.suptitle('The Correlation: Global CO2 Trend vs. Global Temperature Trend', 
             fontsize=16, color='white')
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
legend = ax2.legend(lines_1 + lines_2, labels_1 + labels_2, 
            loc='upper left', facecolor='lightgray', framealpha=0.8, fontsize=10) # Changed facecolor to lightgray for black text visibility
for text in legend.get_texts():
    text.set_color('black')

# Ensure grid and dark theme settings are applied correctly
ax1.grid(True, linestyle='--', alpha=0.5, color='gray') 

plt.tight_layout()
plt.savefig('Plots/co2_vs_temp_correlation.png', dpi=300, bbox_inches='tight', facecolor='#121212')
plt.show()
